In [6]:
import json
import pandas as pd
import re 


#Code could have been under 1/2 methods and condesed a great deal -> I beleive this shows my thought process a bit better

#empty list for data from json
data = []

#loop through json files
for i in range(17):
    # read data
    with open(r"endsong_{}.json".format(i), "r",encoding="utf-8") as f:
        file_data = json.load(f)
    data.extend(file_data)

#create df and append list of data
df = pd.DataFrame.from_records(data)

#remove the "dumb" columns / columns with no track name ("Means that song has been removed or I listened to a podcast 
#not enough data to actually analyze)") format date (ts column)/ and get rid of extra platform info (mostly information that would complicate the analysis and add no benefits)
df = df.drop(columns=['conn_country', 'username', 'user_agent_decrypted', 'incognito_mode', 'offline_timestamp', 'episode_name' , 'episode_show_name', 'spotify_episode_uri'])
df = df.dropna(subset=['master_metadata_track_name'])


df = df.astype({'master_metadata_track_name': 'str'})
df['ts'] = pd.to_datetime(df['ts'], format='%Y-%m-%dT%H:%M:%SZ')

df['platform'] = df['platform'].str.split('(').str[0]
df['platform'] = df['platform'].apply(lambda x: re.findall(r";[^ ;]*", x)[0][1:] if re.findall(r";[^ ;]*", x) else x)
df['platform'] = df['platform'].str.split('.').str[0].str.replace("_", " ")
df['spotify_track_uri'] = df['spotify_track_uri'].str.split(':').str[2]


#write the dataframe to a csv file / used for the first test
df.to_csv(r"endsong_data.csv", index=False)


                        ts               platform  ms_played  \
0      2019-05-26 18:57:17                 iOS 12        384   
1      2021-05-07 01:39:54                 iOS 14      33866   
2      2018-12-31 01:37:21                 iOS 12       1648   
3      2016-07-17 20:29:13                  iOS 9       7379   
4      2017-03-08 07:04:25                 iOS 10      10866   
...                    ...                    ...        ...   
271869 2020-09-01 19:50:43                 iOS 13        767   
271870 2016-12-15 20:36:01                 iOS 10          0   
271871 2021-10-27 18:22:32  Android OS 11 API 30      236480   
271872 2016-06-26 18:25:37                  iOS 8     230634   
271873 2020-10-17 17:40:57                 iOS 14        960   

       ip_addr_decrypted         master_metadata_track_name  \
0          71.104.14.167                    Make It Bun Dem   
1           72.68.39.113              Sunshine Of Your Love   
2         174.200.33.142                  

In [13]:
import requests 
import json


#dictonaries for storing data / used for 
track_artist_mapping = {}
track_song_release_date = {}


#creating batch request (size of 50/request) and inputing track uri and artist id into a dictionary 
#+ adding release date for songs
def batch_requests(url, headers, ids, batch_size=50):
    for i in range(0, len(ids), batch_size):
        ids_batch = ids[i:i+batch_size]
        ids_string = '%2C'.join(ids_batch)
        url_batch = f'{url}?ids={ids_string}'
        response = requests.get(url_batch, headers=headers)
        #check the status code
        if response.status_code != 200:
                print("Error: could not get track information")
                return
        #load data
        data = json.loads(response.text)
        #iterate over json
        for track in data["tracks"]:
            # Get track URI, artist ID, and release date
            track_uri = track["id"]
            artist_id = track["album"]["artists"][0]["id"]
            release_date = track["album"]["release_date"]
            if release_date == "" or release_date == "0000":
                release_date = "NONE" 
                track_song_release_date[track_uri] = release_date
            else:
                release_date = release_date.split("-")[0]
                track_song_release_date[track_uri] = release_date
             # Add track URI and artist ID to dictionary
            track_artist_mapping[track_uri] = artist_id

#api call formatting 
url = 'https://api.spotify.com/v1/tracks'
headers = {
    "Accept": "application/json",
    "Content-Type": "application/json",
    "Authorization": f"Bearer BQDcsujUuXj4mFk_emiHoTPFzVbiyY7E5DQGtHjHFOJnJR9AfCm86bAj8GOpAWbLZb6w0CXROjpnTlamtA2QbGm_CJNJvea0W4i_PNMThjePTU3QCS2wk-qeQ_najfrnpLL2zilyu70v9tGSfv-8s7eb1CyoHQww83EBWtdMMyZgjCnj"}

#get unique tracks so we don't overload spotify api
unique_tracks = df['spotify_track_uri'].unique()
response_list = batch_requests(url, headers, unique_tracks)

#check
print(track_artist_mapping)


{'0mlf3yOGzVnIVxjybDQL3h': '5he5w2lnU9x7JFhnwcekXX', '2K2M0TcglCRLLpFOzKeFZA': '74oJ4qxwOZvX6oSsu1DGnw', '5wBLDkxVvclanSitx5jq8e': '0CEFCo8288kQU7mJi25s6E', '7JGqyqdzijtJeqVRhADFZd': '6J4zegMTYP4nGJSpSDvy49', '3wc0kpZM2lv5UlbA9NFteg': '1HcAkAeL4xf02wzAnl7mIV', '0BWravaqj9rKdOU8RWXhuf': '2Uz58cSxlJgefDaSikxYQ7', '0s3nnoMeVWz3989MkNQiRf': '4MCBfE4596Uoi2O4DtmEMz', '6FqH28raIkXVlcHDUvlEzA': '5X4LWwbUFNzPkEas04uU82', '1x5sYLZiu9r5E43kMlt9f8': '6MDME20pz9RveH9rEXvrOM', '38tKqQuqodiUYOsw6HWyeQ': '25uiPmTg16RbhZWAqwLBy5', '1cURnGaH9mR7YM8SsJHZeS': '6g0Xatzg8vlchoEe21LoeH', '22yRdwQ4OXUlPrLGhF6ebu': '4ZgQDCtRqZlhLswVS6MHN4', '6UjfByV1lDLW0SOVQA4NAi': '3TVXtAsR1Inumwj472S9r4', '4UV1aekV3mXiKqJewTgGRo': '2feDdbD5araYcm6JhFHHw7', '1WQvmGZIMBmapHL3Garw7e': '02kJSzxNuaWGqwubyUba0Z', '2znzyzlAUc8Z1Ikq9aFOcx': '3aGFCoR8xGN6DKwvdzeSja', '34eF4BoV8FPk0uhAAoqU7h': '7jVv8c5Fj3E9VhNjxT4snq', '5Q0LOi7lmEVjpSp0nNGwX6': '6S2OmqARrzebs0tKUEyXyp', '5H4sJFZZjKakgTzyEVmvGX': '5Pb27ujIyYb33zBqVysBkj', '4WmS1Zuled

In [14]:
#making copy of dictionary and mapping values to keys and making an new excel column
track_id_to_artist_id = track_artist_mapping.copy()

df['artist_id'] = df['spotify_track_uri'].map(lambda x: track_id_to_artist_id.get(f'{x}', None))
df['release_date'] = df['spotify_track_uri'].map(lambda x: track_song_release_date.get(f'{x}', None))

#changing the position of the artist_id and release_date column
cols = df.columns.tolist()
cols.remove("artist_id")
cols.remove("release_date")
cols.insert(cols.index("master_metadata_album_artist_name") + 1, "artist_id")
cols.insert(cols.index("artist_id") + 1, "release_date")
df = df[cols]

In [15]:
#another requests that gets the genre of the artists
artist_genre_tracking = {}

def batch_requests(url, headers, ids, batch_size=50):
    for i in range(0, len(ids), batch_size):
        ids_batch = ids[i:i+batch_size]
        ids_string = '%2C'.join(ids_batch)
        url_batch = f'{url}?ids={ids_string}'
        response = requests.get(url_batch, headers=headers)
        #check status
        if response.status_code != 200:
                print("Error: could not get track information")
                return
        #Get JSON data
        data = json.loads(response.text)
        #iterate over json
        for artist in data["artists"]:
            #get artist id and genre
            artist_id = artist["id"]
            genre = artist['genres']
            #spent 30 minutes to realize that i was using if != "" for a list...which is impossible...
            if len(genre) == 0:
                genre = "GNA" 
                artist_genre_tracking[artist_id] = genre
            else:
                genre = genre[0]
                artist_genre_tracking[artist_id] = genre

#api call formatting 
url = 'https://api.spotify.com/v1/artists'
headers = {
    "Accept": "application/json",
    "Content-Type": "application/json",
    "Authorization": f"Bearer BQDcsujUuXj4mFk_emiHoTPFzVbiyY7E5DQGtHjHFOJnJR9AfCm86bAj8GOpAWbLZb6w0CXROjpnTlamtA2QbGm_CJNJvea0W4i_PNMThjePTU3QCS2wk-qeQ_najfrnpLL2zilyu70v9tGSfv-8s7eb1CyoHQww83EBWtdMMyZgjCnj"}

#getting unique artist ids
unique_artists = df['artist_id'].unique()
response_list = batch_requests(url, headers, unique_artists)


In [41]:
#fills genre info into all occurances on the dataframe
df['artist_genre'] = df['artist_id'].map(lambda x: artist_genre_tracking.get(f'{x}', None))

#repostioning column
cols = df.columns.tolist()
cols.remove("artist_genre")
cols.insert(cols.index("master_metadata_album_album_name") + 1, "artist_genre")
df = df[cols]


In [48]:
#had a bit of trouble with this...decided to make a method to find the most frequently occuring genres for each artist
#some artists had pop,rock,and missing values attached to their name, decided to overwrite genres that didn't show up to oftern
def update_genres(df):
    # Group data by artist name
    artist_groups = df.groupby("master_metadata_album_artist_name")

    # Iterate over
    for name, group in artist_groups:
        # Get the most common genre
        most_common_genres = group["artist_genre"].mode()
        # check if there are more than one genres
        if len(most_common_genres) > 1:
            most_common_genre = most_common_genres[1] if most_common_genres[0] == "GNA" else most_common_genres[0]
        else:
            most_common_genre = most_common_genres[0]
        # Replace all genres for the artist, with the most common genre
        df.loc[group.index, "artist_genre"] = most_common_genre
    return df
df = update_genres(df)

#create csv for data analysis
df.to_csv(r"final_data.csv", index=False)